In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import ast
import matplotlib.dates as mdates
import sumolib
import osmnx as ox

import sys
import os
import glob
import random
import logging
import subprocess
import numpy as np

from fastsim import simdrive, vehicle, cycle
from fastsim import parameters as params

import traci
import time
import csv

import xml.etree.ElementTree as ET
import xml.dom.minidom

from datetime import datetime, timedelta

import utils.simulation as sim 
from utils.featureExtraction import feature_extraction
# postprocessing to edge level
from utils.postprocessing import process_edge_gdf, process_one_file, edge_list_to_node_list

2024-03-06 20:41:56.180 | calibration.py:38 | WARNING: No module named 'pymoo'
Try running `pip install pymoo==0.6.0.1` to use all features in `fastsim.calibration`


In [2]:
number_of_trips = 5

data_file_folder = "data"
results_file_folder = "results"

edge_level_data_folder = os.path.join(data_file_folder, "processed/synthetic")
results_data_file_folder = os.path.join(data_file_folder, "features/synthetic")

os.makedirs(edge_level_data_folder, exist_ok=True)  # Create the destination folder if it doesn't exist
os.makedirs(results_data_file_folder, exist_ok=True) 

In [3]:
# Set SUMO_HOME; revise it according to the path to the site-packages folder of SUMO  
os.environ['PATH'] += ":/home/shekhars/yang7492/.conda/envs/syntheticData/lib/python3.8/site-packages/sumo/bin"
os.environ['SUMO_HOME'] = '/home/shekhars/yang7492/.conda/envs/syntheticData/lib/python3.8/site-packages/sumo'

In [4]:
# load openstreet map:
osm_file_path = os.path.join(data_file_folder, "maps/minneapolis.graphml")
osmnx_net = ox.io.load_graphml(osm_file_path)
node_gdf, edge_gdf = ox.utils_graph.graph_to_gdfs(osmnx_net)

In [5]:
# load sumo map:
net_file = os.path.join(data_file_folder, "Minneapolis.net.xml")
sumo_net = sumolib.net.readNet(net_file)

In [6]:
# generate k trip files
node_ids = node_gdf.index.tolist()  # List of node IDs

origins = []
destinations = []

for _ in range(number_of_trips):
    origin, destination = random.sample(node_ids, 2)  # Sample two unique nodes
    origins.append(origin)
    destinations.append(destination)

# random node paths
random_paths = ox.routing.shortest_path(osmnx_net, origins, destinations, weight='length', cpus=1)

In [7]:
# Convert osmnx paths to sumo paths
all_routes_info, vehicle_ids = sim.osmnx_paths_to_sumo_routes(random_paths, sumo_net)   

In [8]:
# SUMO simulation configuration

# After collecting routes_info from all files and trips
route_file = os.path.join(data_file_folder, "incompelete_routes.xml")
sim.save_incomplete_routes_to_xml(all_routes_info, route_file)

complete_route_file = os.path.join(data_file_folder, "complete_routes.rou.xml")
sim.complete_routes(route_file, net_file, complete_route_file)

begin = 0
end = 14400 # maximum simulated travel time 
step_length = 1
file_name_config = "sumo.sumocfg"
sim.save_sumo_config_to_file(net_file, complete_route_file, begin, end, step_length, file_name_config)

# sumo simulation
velocity_data, edgeSeq_data = sim.sumo_simulation(file_name_config, vehicle_ids)

 Retrying in 1 seconds
Step #5302.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 5 ACT 0 BUF 0)                    tep #4500.00 (3ms ~= 333.33*RT, ~333.33UPS, TraCI: 1ms, vehicles TOT 5 ACT 1 BUF 0)      


In [9]:
# Generate synthetic vehicle type
# 62 predefined vehicle types in FASTSim: https://github.com/NREL/fastsim/blob/fastsim-2/python/fastsim/resources/FASTSim_py_veh_db.csv
vehicle_type = np.random.randint(1, 27, number_of_trips)
# vehicle_type = 0 -> the predefined Murphy Heavey Duty Truck. Refer to cumstomize_veh() in simultation.py
# vehicle_type = [0 for _ in range(number_of_trips)]

# generte a dataframe of the synthetic data based on SUMO results
csv_file = sim.generate_synthetic_csv(random_paths, velocity_data, edgeSeq_data, vehicle_type, edge_gdf, sumo_net)

In [10]:
csv_file.head()

,trip_start_time,trip_end_time,travel_time,velocity_profile,weight,total_fuel,ambTemperature,trajectory,matched_path,coordinate_id,road_id,vehicle_type
0,2023-10-30 16:51:58.547468,2023-10-30 17:00:46.547468,528,"[0.0, 1.9189507206669076, 3.641124780732207, 5...",[1959.8983587886976],[93.60458479816482],4,"[(44.78809, -93.462495)]","[(188494206, 188494205, 0), (188494205, 188501...",[0],"[(188494206, 188494205, 0), (188494206, 188494...",16
1,2024-02-13 09:29:24.547482,2024-02-13 09:45:37.547482,973,"[0.0, 1.6141484214691446, 3.9632478857645768, ...",[1882.8506869071566],[82.18218246177022],-8,"[(44.78809, -93.462495)]","[(186823394, 186797287, 0), (186797287, 186797...",[0],"[(186823394, 186797287, 0), (186823394, 186797...",14
2,2023-10-06 12:52:09.547485,2023-10-06 13:31:36.547485,2367,"[0.0, 2.215580755379051, 4.730005498952232, 7....",[1317.5469983063322],[77.08542457615346],11,"[(44.78809, -93.462495)]","[(188577573, 188577576, 0), (188577576, 188564...",[0],"[(188577573, 188577576, 0), (188577573, 188577...",24
3,2023-06-02 01:22:29.547488,2023-06-02 02:16:45.547488,3256,"[0.0, 2.0975293326890094, 4.486730313114823, 5...",[1840.913513682106],[93.29763058958196],9,"[(44.78809, -93.462495)]","[(186515281, 186481805, 0), (186481805, 186496...",[0],"[(186515281, 186481805, 0), (186515281, 186481...",1
4,2023-09-11 03:17:45.547491,2023-09-11 04:46:06.547491,5301,"[0.0, 2.522357051051222, 4.150746611505747, 5....",[1848.282544802568],[84.1938416081558],-8,"[(44.78809, -93.462495)]","[(186798717, 186798709, 0), (186798709, 186798...",[0],"[(186798717, 186798709, 0), (186798717, 186798...",2


In [11]:
# Process the data (trip level) and save to the destination folder
simulated_data = sim.fastsim(csv_file, velocity_data, edgeSeq_data, data_file_folder)

2024-03-06 20:51:18.769 | simdrive.py:2262 | WARNING: problem detected with conservation of energy; energy audit error: 0.00739
2024-03-06 20:51:18.771 | simdrive.py:2298 | WARNING: trace miss speed 1.03733 m/s exceeds tolerance of 1.0 m/s
2024-03-06 20:51:19.261 | simdrive.py:2262 | WARNING: problem detected with conservation of energy; energy audit error: 0.00919
2024-03-06 20:51:19.263 | simdrive.py:2298 | WARNING: trace miss speed 1.03733 m/s exceeds tolerance of 1.0 m/s
2024-03-06 20:51:19.776 | simdrive.py:2262 | WARNING: problem detected with conservation of energy; energy audit error: 0.00919
2024-03-06 20:51:19.778 | simdrive.py:2298 | WARNING: trace miss speed 1.03733 m/s exceeds tolerance of 1.0 m/s
2024-03-06 20:51:20.158 | simdrive.py:2280 | WARNING: trace miss distance fraction 0.00682 exceeds tolerance of 0.001
2024-03-06 20:51:20.160 | simdrive.py:2298 | WARNING: trace miss speed 4.92621 m/s exceeds tolerance of 1.0 m/s
2024-03-06 20:51:20.725 | simdrive.py:2280 | WARNI

In [12]:
# Convert space-separated strings to lists of floats
float_list_columns = ['fastsim_velocity', 'fastsim_power', 'sumo_velocity']
for column in float_list_columns:
    simulated_data[column] = simulated_data[column].apply(lambda x: [float(i) for i in x.split()])

# Convert space-separated strings to lists
simulated_data['sumo_path'] = simulated_data['sumo_path'].str.split()

In [13]:
# conver the processed data to edge level
edge_gdf = process_edge_gdf(edge_gdf, node_gdf, results_file_folder)
df_edge = process_one_file(simulated_data, edge_gdf, sumo_net)
# Save the processed DataFrame to a new CSV in the output folder
edge_level_output_file_name = os.path.join(edge_level_data_folder, "synthetic.csv")
df_edge.to_csv(edge_level_output_file_name, index=False)
print(f"Saved processed data to {edge_level_output_file_name}")

Elevation total batches to fetch : 14
Elevation processing batch 1 of 14
Elevation processing batch 2 of 14
Elevation processing batch 3 of 14
Elevation processing batch 4 of 14
Elevation processing batch 5 of 14
Elevation processing batch 6 of 14
Elevation processing batch 7 of 14
Elevation processing batch 8 of 14
Elevation processing batch 9 of 14
Elevation processing batch 10 of 14
Elevation processing batch 11 of 14
Elevation processing batch 12 of 14
Elevation processing batch 13 of 14
Elevation processing batch 14 of 14
700
Saved processed data to data/processed/synthetic/synthetic.csv


In [14]:
edge_level_file_pattern = os.path.join(edge_level_data_folder, "*.csv")
synthetic_data_flg = True
feature_extraction(data_file_folder, results_file_folder, edge_level_file_pattern, results_data_file_folder, synthetic_data_flg)
print(f"Saved extracted feature data to {results_data_file_folder}")